# Loadout

In [3]:
from abc import ABCMeta, abstractmethod, ABC
from pathlib import Path
from uuid import uuid4
from typing import Optional, Callable, List
from typing import TYPE_CHECKING, Union, List
import inspect
import asyncio

from ruamel.yaml import YAML

import panel as pn
from panel.viewable import Viewer

import param
from param import rx, Parameterized

from langchain_core.language_models import BaseLanguageModel
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma # Useless
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage

import chromadb
from chromadb.api.client import Client
from chromadb.api.models.Collection import Collection

import tiktoken

from dotenv import load_dotenv


###################Other Thangs#######################################
import sys
sys.path.append("..")

from pyllments.config import BASE_DIR

load_dotenv()
pn.extension()

In [132]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Framework Construction

## Ports

In [134]:
%%writefile /workspaces/pyllments/pyllments/ports/ports.py
# from typing import TYPE_CHECKING, Union, List
# import inspect
# import warnings

# import param

# from pyllments.base import Payload


class Port(param.Parameterized):
    payload = param.ClassSelector(class_=Payload)
    containing_element = param.Parameter(default=None, precedence=-1)
    connected_elements = param.List()

    def __init__(self, **params):
        super().__init__(**params)
        # Set as attribute and clear parameter to avoid circular param __repl__
        self._containing_element = self.containing_element
        self.containing_element = None


class InputPort(Port):
    subject_ports = param.List(item_type=Port)

    unpack_payload_callback = param.Callable(doc="""
        The callback used to unpack the payload - has payload as its only argument.
        Unpacks the payload and connects it to the element's model
        """)

    def receive(self, payload):
        ### Just TESTING
        self.payload = payload
        if not self.unpack_payload_callback:
            raise ValueError('unpack_payload_callback must be set')
        self.unpack_payload_callback(payload)

class OutputPort(Port):
    """
    Handles the intake of data and packing into a payload and
    is meant to connect to an InputPort in order to emit the packed payload
    """
    # Add a watcher IF updating of required_items becomes necessary
    required_items = param.List(item_type=Union[str, tuple[str, type]], doc="""
        Type checking automatically enabled if (name, type) tuple is passed""")

    emit_when_ready = param.Boolean(default=True, doc="""
        If true, the payload will be emitted when the required items are staged""")

    emit_ready = param.Boolean(default=False, doc="""
        True when the required items have been staged and the payload can be emitted""")

    infer_from_callback = param.Boolean(default=True, doc="""
        If true, infers the required items from pack_payload_callback
        and required_items is set to None""")

    observer_ports = param.List(item_type=Port, doc="""
        The connected InputPorts which emit() will contact""")

    pack_payload_callback = param.Callable(default=None, doc="""
        The callback used to create the payload. When used in conjunction with
        infer_required_items == True, an annotated callback can replace passing in
        a payload and required_items while enabling type-checking.
        Kwargs, their types, and the return type are required annotations.""")
        
    staged_items = param.List(item_type=str, doc="""
        The items that have been staged and are awaiting emission""")
    
    type_checking = param.Boolean(default=False, doc="""
        If true, type-checking is enabled""")

    def __init__(self, **params: param.Parameter):
        super().__init__(**params)

        # In the case when pack_payload_callback is passed and so is infer_from_callback
        if self.pack_payload_callback and self.infer_from_callback:
            if (self.infer_from_callback and
                not inspect.getfullargspec(self.pack_payload_callback).annotations):

                raise ValueError("""
                    pack_payload_callback must have annotations if infer_from_callback
                    is False""")
            if self.payload is not None:
                warnings.warn("""payload will be overridden with the return type of 
                    pack_payload_callback""")
            self.type_checking = True
            annotations = inspect.getfullargspec(self.pack_payload_callback).annotations
            self.param.payload.class_ = annotations.pop('return')
            self.required_items =  tuple(annotations.items())
        # In the case of desired type-checking
        elif self.required_items and (isinstance(self.required_items[0], tuple)):
            self.type_checking = True
            for item in self.required_items:
                self.param.add_parameter(
                    item[0],
                    param.ClassSelector(class_=[item[1]])
                    )
        # In case of no type-checking but required_items specified
        elif str and (not self.required_items):
            for item in self.required_items:
                self.param.add_parameter(item, param.Parameter())
        elif not self.required_items:
            self.type_checking = False

    def connect(self, other: InputPort):
        """Connects self and the other InputPort"""
        if not isinstance(other, InputPort):
            raise ValueError('Can only connect OutputPorts to InputPorts')
        if not isinstance(other.payload, type(self.payload)):
            raise ValueError('InputPort and OutputPort payload types must match')
        
        self.observer_ports.append(other)
        self.connected_elements.append(other._containing_element)
        other.connected_elements.append(self._containing_element)
    
    def stage(self, **kwargs: param.Parameter):
        """Stages the values within the port before packing"""
        if self.type_checking:
            for name, value in kwargs.items():
                class_ = self.param[name].class_
                if isinstance(value, class_):
                    self.staged_items.append(name)
                    self.param[name] = value
            # Compare equality of staged_items and required_items
            if set(self.staged_items) == set(item[0] for item in self.required_items):
                self.emit_ready = True
        else:
            for name, value in kwargs.items():
                self.staged_items.append(name)
                self.param[name] = value
            if set(self.staged_items) == set(item[0] for item in self.required_items):
                self.emit_ready = True
        if self.emit_when_ready and self.emit_ready:
            self.emit()

    def emit(self):
        """Packs the payload and emits it to all registered observers"""
        if not self.emit_ready:
            raise Exception('Staged items do not match required items')
        else:
            self.payload = self.pack_payload()
        for port in self.observer_ports:
            port.receive(self.payload)
        # For returning the payload to the caller 
        return self.payload
    
    def stage_and_emit(self, **kwargs):
        """Stages the payload and emits it - All required params need be present"""
        self.stage(**kwargs)
        self.emit()

    def pack_payload(self):
        if self.pack_payload_callback:
            staged_dict = {}
            for item in self.staged_items:
                staged_dict[item] = getattr(self, item)
            return self.pack_payload_callback(**staged_dict)
        else:
            raise ValueError('pack_payload_callback must be set')
            #TODO Implement default callback based on the payload type

    def __gt__(self, other):
        """Implements self.connect(other) through el1.some_input > el2.some_output"""
        self.connect(other)
        


class InputPorts(param.Parameterized):
    containing_element = param.Parameter(precedence=-1)

    def __init__(self, **params):
        self._containing_element = self.containing_element
        self.containing_element = None

    def add(self, name, payload_type, **kwargs): 
        input_port = InputPort(
            payload=payload_type(), # Overridden if pack_payload_callback is passed
            **kwargs
            )

        self.param.add_parameter(name, param.Parameter(input_port))
        return input_port

class OutputPorts(param.Parameterized):
    containing_element = param.Parameter(precedence=-1)

    def __init__(self, **params):
        self._containing_element = self.containing_element
        self.containing_element = None

    def add(self, name, payload_type=None, **kwargs):
        if payload_type:
            if kwargs.get('payload'):
                warnings.warn(
                    "payload_type will override the payload argument if both are provided." 
                )
            kwargs['payload'] = payload_type()
        output_port = OutputPort(**kwargs)
        self.param.add_parameter(name, param.Parameter(output_port))
        return output_port

class Ports(param.Parameterized):
    """Keeps track of InputPorts and OutputPorts and handles their creation"""
    input = param.ClassSelector(class_=InputPorts, default=InputPorts())
    output = param.ClassSelector(class_=OutputPorts, default=OutputPorts())
    containing_element = param.Parameter(precedence=-1)

    def __init__(self, **params):
        super().__init__(**params)
        self._containing_element = self.containing_element
        self.containing_element = None

    def add_input(self, **kwargs): 
        self.input.add(
            containing_element=self._containing_element,
            **kwargs
            )
    
    def add_output(self, **kwargs):
        self.output.add(
            containing_element=self._containing_element,
            **kwargs
            )


Overwriting /workspaces/pyllments/pyllments/ports/ports.py


In [114]:
port = InputPort(containing_element='some_element')

In [131]:
def test_port_connection():
    """
    Connects the output port of one element to the input port of another
    and tests whether the connected elements in each port are indeed connected
    """
    el1 = Element()
    el2 = Element()

    el1.ports.add_output(name='some_output', payload_type=Payload)
    el2.ports.add_input(name='some_input', payload_type=Payload)
    print('payload of el1: ', el1.ports.output.some_output.payload, 
        '\npayload of el2: ', el2.ports.input.some_input.payload)

    el1.ports.output.some_output > el2.ports.input.some_input

    assert el1.ports.output.some_output.connected_elements[0] is el2
    assert el2.ports.input.some_input.connected_elements[0] is el1

test_port_connection()


payload of el1:  <Payload Payload00379> 
payload of el2:  <Payload Payload00381>


## Base

In [5]:
# %%writefile /workspaces/pyllments/pyllments/base/model.py
# import param

class Model(param.Parameterized):
    def __init__(self, **params):
        super().__init__(**params)

In [8]:
# %%writefile /workspaces/pyllments/pyllments/base/payload.py
# import param

class Payload(param.Parameterized):
    pass

In [129]:
# %%writefile /workspaces/pyllments/pyllments/base/element.py
# import param

# from pyllments.ports import InputPorts, OutputPorts

class Element(param.Parameterized):
    model = param.ClassSelector(class_=Model, default=None)
    ports = param.ClassSelector(class_=Ports)

    def __init__(self, **params):
        super().__init__(**params)
        self.ports = Ports(containing_element=self)


In [99]:
# %%writefile /workspaces/pyllments/pyllments/base/listing.py
# import param

class Listing(param.Parameterized):
    def __init__(self, **params):
        super().__init__(**params)

Overwriting /workspaces/pyllments/pyllments/base/listing.py


## Payload

### Message Payload

In [9]:
# %%writefile /workspaces/pyllments/pyllments/payloads/message_payload/message_model.py
# import param

# from pyllments.base import Model

class MessageModel(Model):
    message = param.String(default=None, per_instance=True)
    message_type = param.String(default=None, per_instance=True)

    # payload_pack

In [14]:
# %%writefile /workspaces/pyllments/pyllments/payloads/message/message_payload.py
# import param
# import panel as pn

# from pyllments.base import Payload
# from pyllments.payloads.message.message_model import MessageModel

class MessagePayload(Payload):
    model = param.ClassSelector(class_=MessageModel, is_instance=True)
    message_type = param.String(default=None, per_instance=True)
    message_view = param.ClassSelector(class_=pn.pane.Markdown, is_instance=True)

    def create_message_view(self, **kwargs):
        self.message_view = pn.pane.Markdown(self.message)
        return self.message_view

## ChatInterface

### ChatInterfaceModel

In [6]:
%%writefile /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_model.py
import param

from pyllments.base import Model
from pyllments.payloads.message import MessagePayload

class ChatInterfaceModel(Model):
    message_list = param.List(default=[], per_instance=True)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.param.add_parameter(
            'created_message',
            param.ClassSelector(class_=MessagePayload)
            )
        self.message_list.append(('human', 'what is the capital of the moon?'))

Overwriting /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_model.py


### ChatInterfaceElement

In [2]:
# %%writefile /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_element.py
# import panel as pn
# import param

# from pyllments.base import Element, Model
# from pyllments.elements.chat_interface import ChatInterfaceModel

class ChatInterfaceElement(Element):
    '''Responsible for creating the chat feed, input, and send button views'''
    model = param.ClassSelector(class_=Model, default=ChatInterfaceModel())

    chatfeed_view = param.ClassSelector(class_=pn.chat.ChatFeed, is_instance=True)
    chat_input_view = param.ClassSelector(class_=pn.chat.ChatAreaInput, is_instance=True)
    send_button_view = param.ClassSelector(class_=pn.widgets.Button, is_instance=True)

    def create_chatfeed_view(self, **kwargs):
        self.chatfeed_view = pn.chat.ChatFeed(**kwargs)
        return self.chatfeed_view
    
    def create_chat_input_view(self, **kwargs):
        self.chat_input_view = pn.chat.ChatAreaInput(**kwargs)
        return self.chat_input_view

    def create_send_button_view(self, **kwargs):
        self.send_button_view = pn.widgets.Button(**kwargs)
        return self.send_button_view

    def _on_send(self, event):
        self.chat_interface_model.message_list.append(('human', self.chat_input_view.value))
        self.chat_input_view.value = ''
        self.chatfeed_view.update()

Overwriting /workspaces/pyllments/pyllments/elements/chat_interface/chat_interface_element.py


In [3]:
from pyllments.elements.chat_interface import ChatInterfaceElement

## LLM Chat

### LLM Chat Model

In [4]:
# %%writefile /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_model.py
# import param

# from langchain_core.language_models import BaseLanguageModel
# from langchain_openai import ChatOpenAI

# from pyllments.base import Model

class LLMChatModel(Model):
    model_class = param.ClassSelector(
        class_=BaseLanguageModel,
        is_instance=False,
        doc='Class of a LangChain Chat Model',
        default=ChatOpenAI
    )
    model_args = param.ClassSelector(
        class_=(dict, list), #TODO: May replace with dict, as the elements will the logic for view creation
        doc='''Takes a dictionary of arguments to pass to expose and send to the
        model class. If you set a None value as the key, the argument will be exposed,
        with the default value set. Passing a list is the same as passing a dict
        with all values set to None.''',
        is_instance=True,
        default={}
        )

    model = param.ClassSelector(class_=BaseLanguageModel, default=None, pickle_default_value=False)

    def __init__(self, **params):
        super().__init__(**params)

        self._set_params()
        self._create_model()

    def _set_params(self):
        """Sets specified model_args as params of the object"""
        if self.model_args:
            for arg, val in self.model_args.items():
                if arg in self.model_class.__fields__:
                    if val is None:
                        default = self.model_class.__fields__[arg].default
                        self.model_args[arg] = default
                        self.param.add_parameter(arg, param.Parameter(default, per_instance=True))
                    else:
                        self.param.add_parameter(arg, param.Parameter(val, per_instance=True))
                    # self.model_args_list.append(arg)
                else:
                    raise ValueError(f"'{arg}' is missing from the model class's signature")
                self.param.watch(self._create_model, [*self.model_args.keys()])


    def _create_model(self, event=None):
        """Creates the model instance on init and when any of the parameters change"""
        arg_vals = {arg: self.param.values()[arg] for arg in self.model_args.keys()}
        self.model = self.model_class(**arg_vals)

    def stream(self, messages):
        # TODO: Set it up to be async
        return self.model.stream(messages)

Writing /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_model.py


In [1]:
from pyllments.elements.llm_chat import LLMChatModel

### LLM Chat Element

In [6]:
# %%writefile /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_element.py
# import param
# import panel as pn

# from pyllments.base import Element
# from pyllments.elements.llm_chat import LLMChatModel

class LLMChatElement(Element):
    model = param.ClassSelector(class_=LLMChatModel)

    temperature = param.Number(default=0.5)
    temperature_view = param.ClassSelector(class_=pn.widgets.FloatSlider, is_instance=True)

    def __init__(self, **params):
        super().__init__(**params)

    def create_temperature_view(self, **kwargs):
        self.temperature_view = pn.widgets.FloatSlider(**kwargs)
        return self.temperature_view

Writing /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_element.py


In [9]:
from pyllments.elements.llm_chat import LLMChatElement

### LLM Chat Listing

In [10]:
%%writefile /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_listing.py
import param
import panel as pn
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

from pyllments.base import Listing
from pyllments.elements.llm_chat import LLMChatElement, LLMChatModel

class LLMChatListing(Listing):
    selector = param.ClassSelector(class_=pn.widgets.Select, is_instance=True)
    llm_chat_element = param.ClassSelector(class_=LLMChatElement, is_instance=True)
    chat_model_dict = param.Dict(default={
        'gpt-3.5-turbo': ChatOpenAI,
        'gpt-4o': ChatOpenAI,
        'claude-3-opus-20240229': ChatAnthropic,
    })
    model_args = param.Dict(default={
        'temperature': 0.5
    })
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        default_model_name = next(iter(self.chat_model_dict))
        self.create_llm_chat_element(default_model_name)

    def create_llm_chat_element(self, chat_model_name):
        """Creates and sets the chat model element"""
        llm_chat_model = LLMChatModel(
            model_class=self.chat_model_dict[chat_model_name],
            model_args=self.model_args | {'model_name': chat_model_name}
            )
        self.llm_chat_element = LLMChatElement(llm_chat_model=llm_chat_model)
        return self.llm_chat_element

    def create_selector(self):
        self.selector = pn.widgets.Select(options=list(self.chat_model_dict.keys()))
        self.selector.param.watch(self._on_selector_change, 'value')
        return self.selector
    
    def _on_selector_change(self, event):
        """Callback method triggered when the selector value changes"""
        # print(event)
        self.create_llm_chat_element(event.new)


Writing /workspaces/pyllments/pyllments/elements/llm_chat/llm_chat_listing.py


In [1]:
from pyllments.elements.llm_chat import LLMChatListing

In [245]:
chat_listing = LLMChatListing()

chat_listing.create_selector()
chat_listing.selector

BokehModel(combine_events=True, render_bundle={'docs_json': {'2d711ee2-5b70-4054-adda-f61d0e46ab54': {'version…

### Chat

In [3]:
class ConversationHistory:
    pass

class ContextHandler:
    pass

class RetrieverStore:
    pass

class ReplyHandler:
    pass


In [221]:
ChatInterfaceElement.param

Name,Default,Type,Range
chat_input_view,None,ClassSelector,nullable ChatAreaInput
chatfeed_view,None,ClassSelector,nullable ChatFeed
name,'ChatInterfaceElement',String,nullable constant


In [55]:
# Model layer
class Chat(param.Parameterized):
    conversation_history = param.ClassSelector(
       class_=ConversationHistory,
       default=None,
       doc="Message handler between AI and User - could contain source messages as well")
    context_history = param.ClassSelector(
        class_=ContextHandler,
        default=None,
        is_instance=True,
        doc="Context handler")
    use_context = param.Boolean(
        default=False,
        doc="Whether to use context from the context handler"
    )
    retriever_store = param.ClassSelector(
        class_=RetrieverStore,
        default=None,
        is_instance=True,
        doc="Retriever store")
    retrieval = param.ClassSelector(
        class_=BaseRAG,
        default=None,
        is_instance=True,
        doc="RAG handler")
    reply = param.ClassSelector(
        class_=ReplyHandler,
        default=None,
        is_instance=True,
        doc="Last Generated Reply")
    # sources = param.List(default=[], doc="List of sources to retrieve")
    prompt_template = param.ClassSelector(
        class_=ChatPromptTemplate,
        is_instance=True,
        doc="Prompt template for the chat model")
    system_message = param.String(
        default="You're a helpful assistant",
        doc="System message for the prompt"
    )
    chat_model = param.ClassSelector(
        class_=LLMChatModel,
        is_instance=True,
        doc="Chat model"
    )

    
    def generate_reply(self, message, astream=True):
        """"""
        # sources = self.rag.get_sources(message) 
        # context = self.context_history.get_context(sources)
        messages = [SystemMessage(self.system_message), HumanMessage(message)]
        if self.use_context:
            messages.insert(1, 'history goes here')
        prompt_template = ChatPromptTemplate.from_messages(messages)
        chain = prompt_template | self.chat_model.model
        if astream:
            stream = chain.astream({'message': message})
            return stream
        else:
            return chain.stream({'message': message})

In [56]:
openai_chat_model = ChatOpenAI
llm_chat_model = LLMChatModel(model_class=ChatOpenAI, model_args={'model_name': 'gpt-3.5-turbo'})

In [57]:
chat = Chat(
    chat_model=llm_chat_model,
    system_message='You are a posh model that answers questions verbosely.')

In [173]:
reply_stream = chat.generate_reply('what is the capital of the moon?', astream=True)
md = pn.pane.Markdown('# ')
cm = pn.chat.ChatMessage(md, user="Wise guy", avatar="🤓")
cm

BokehModel(combine_events=True, render_bundle={'docs_json': {'7b91a5e7-4a10-42fc-83e8-9b1a58f2582e': {'version…

In [175]:
async for token in reply_stream:
    md.object += f' {token.content}'

In [90]:
type(reply_stream)

async_generator

In [170]:
md = pn.pane.Markdown('# HI')
cm = pn.chat.ChatMessage(md, user="Wise guy", avatar="🤓")
cm

BokehModel(combine_events=True, render_bundle={'docs_json': {'8c45e048-8a01-4e98-8bbf-f27c2bda892b': {'version…

In [10]:
# View Layer
class ChatView(Viewer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.chat_input = pn.chat.ChatAreaInput()
        self.chat_feed = pn.chat.ChatFeed()
        self.chat_send_button = pn.widgets.Button(name='Send')
        self.chat_retrieve_switch = pn.widgets.Switch(name='Retrieve')
        self.chat_list = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.corpus_selector = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        # self.CorpusParametersView = CorpusParametersView()
        
    
    def __panel__(self):
        return pn.Column(self.chat_feed, self.chat_input)

In [ ]:
class ChatController(param.Parameterized):
    chat_view = param.

In [ ]:
class CorpusManager(param.Parameterized):
    corpus_list = param.List(default=[])
    selected_corpus = param.String()
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        corpus_list = self.get_corpora()
    
    def get_corpora(self):
        corpora_path = BASE_DIR / 'corpora'
        return [f.name for f in corpora_path.iterdir() if f.is_dir()]
        



## View Layer

In [11]:
pn.panel(ChatView())

BokehModel(combine_events=True, render_bundle={'docs_json': {'9d07223d-8e03-49c3-a46f-cdfb05e194f2': {'version…

In [ ]:
class RetrieverCreationView(Viewer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.retriever_type = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.corpus_selector = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.CorpusParametersView = CorpusParametersView()
        
    def __panel__(self):
        return pn.Column(self.retriever_type, self.corpus_selector, self.CorpusParametersView)



## Controller Layer

In [ ]:
# Controller Layer
class ChatController(param.Parameterized):
    view = param.ClassSelector(class_=ChatView, is_instance=True)
    model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)

    def __panel__(self):
        return self.view()

In [26]:
pn.panel(ChatInterface())

BokehModel(combine_events=True, render_bundle={'docs_json': {'c0cc8570-7eb7-4dcb-9412-cd85afe5b670': {'version…

In [45]:
def on_click(event):
    print("ChatMessage clicked!")

message = pn.chat.ChatMessage(pn.Row(pn.widgets.Button(name='click me', height=50), pn.pane.Markdown('# some test'), pn.widgets.Button(name='click me', height=50),  pn.pane.Markdown('# some test')))
serve(message)

Launching server at http://localhost:35594


In [50]:
chat_feed.send('interior crocadile alligator')

BokehModel(combine_events=True, render_bundle={'docs_json': {'682c0bce-e79a-4b7f-9753-e8290db425be': {'version…

In [48]:
import asyncio

In [65]:
chat_feed = pn.chat.ChatFeed()

# creates a new message
message = chat_feed.stream("Hello", user="Aspiring User", avatar="🤓")
chat_feed

BokehModel(combine_events=True, render_bundle={'docs_json': {'e465d4fe-140f-4671-8dac-d50f2ef73d36': {'version…

In [67]:
# streams (appends) to the previous message
message = chat_feed.stream(
    'what is the capital of France?',
    user="reeeee",
    avatar="🤓",
    message=message,
)

In [64]:
# streams (appends) to the previous message
message = chat_feed.stream(
    pn.pane.Markdown('is it me'),
    user="reeeee",
    avatar="🤓",
    message=message,
)

# Project initialization scripts

In [ ]:
# cli.py

import argparse

def create_project():
    """Establishes a new project directory structure and config"""
    parser = argparse.ArgumentParser(description='Initialize a new project')
    parser.add_argument('name', type=str, help='Name of the project')
    parser.add_argument('--include-examples', action='store_true', help='Include examples in the project')
    args = parser.parse_args()

    project_path = Path.cwd() / args.name
    project_path.mkdir(parents=True, exist_ok=True)
    
    (project_path / "corpora").mkdir(exist_ok=True)
    (project_path / "retrievers").mkdir(exist_ok=True)
    (project_path / "memory").mkdir(exist_ok=True)
    (project_path / "files").mkdir(exist_ok=True)
    (project_path / "retrieval_database").mkdir(exist_ok=True)

    with open(project_path / "project.yaml", 'w') as file:
        yaml = YAML()
        data = {'include_examples': args.include_examples}
        yaml.dump(data, file)



# App Creation

In [ ]:
def app_run():
    

    pn.serve(app)

